In [1]:
import azure, json, os, requests
import time
from requests import get, post
from dotenv import load_dotenv

In [2]:
load_dotenv(verbose=True)

True

In [3]:
########### Python Form Recognizer Labeled Async Train #############

# Endpoint URL
endpoint = os.getenv("ENDPOINT") 
post_url = endpoint + r"/formrecognizer/v2.0-preview/custom/models"
source = os.getenv("SAS_URL")
prefix = ""
includeSubFolders = False
useLabelFile = False

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': os.getenv("KEY"),
}

body = 	{
    "source": source,
    "sourceFilter": {
        "prefix": prefix,
        "includeSubFolders": includeSubFolders
    },
    "useLabelFile": useLabelFile
}

try:
    resp = post(url = post_url, json = body, headers = headers)
    if resp.status_code != 201:
        print("POST model failed (%s):\n%s" % (resp.status_code, json.dumps(resp.json())))
        quit()
    print("POST model succeeded:\n%s" % resp.headers)
    get_url = resp.headers["location"]
except Exception as e:
    print("POST model failed:\n%s" % str(e))
    quit()

POST model succeeded:
{'Content-Length': '0', 'Location': 'https://formrecognisertest-kawar.cognitiveservices.azure.com/formrecognizer/v2.0-preview/custom/models/0bba6506-c055-4518-889e-516c588c2ccd', 'x-envoy-upstream-service-time': '185', 'apim-request-id': '81a4592a-b64a-43ea-88e8-152bb146c70b', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'Date': 'Mon, 06 Apr 2020 23:37:45 GMT'}


In [4]:
n_tries = 15
n_try = 0
wait_sec = 5
max_wait_sec = 60
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = headers)
        resp_json = resp.json()
        if resp.status_code != 200:
            print("GET model failed (%s):\n%s" % (resp.status_code, json.dumps(resp_json)))
            quit()
        model_status = resp_json["modelInfo"]["status"]
        if model_status == "ready":
            print("Training succeeded:\n%s" % json.dumps(resp_json, indent=4, sort_keys=True))
            modelID = resp_json["modelInfo"]["modelId"]
            print("ModelID:\n%s" % modelID)
            break
        if model_status == "invalid":
            print("Training failed. Model is invalid:\n%s" % json.dumps(resp_json))
            quit()
        # Training still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1
        wait_sec = min(2*wait_sec, max_wait_sec)     
    except Exception as e:
        msg = "GET model failed:\n%s" % str(e)
        print(msg)
        quit()
print("Train operation did not complete within the allocated time.")

Training succeeded:
{
    "modelInfo": {
        "createdDateTime": "2020-04-06T23:37:45Z",
        "lastUpdatedDateTime": "2020-04-06T23:38:02Z",
        "modelId": "0bba6506-c055-4518-889e-516c588c2ccd",
        "status": "ready"
    },
    "trainResult": {
        "errors": [],
        "trainingDocuments": [
            {
                "documentName": "EmptyPage1.png",
                "errors": [],
                "pages": 1,
                "status": "succeeded"
            },
            {
                "documentName": "Form1.png",
                "errors": [],
                "pages": 1,
                "status": "succeeded"
            },
            {
                "documentName": "Form2.png",
                "errors": [],
                "pages": 1,
                "status": "succeeded"
            }
        ]
    }
}
ModelID:
0bba6506-c055-4518-889e-516c588c2ccd
Train operation did not complete within the allocated time.


In [5]:
########### Python Form Recognizer Async Analyze #############
# Endpoint URL
#endpoint = r"<endpoint>"
apim_key = os.getenv("KEY")
#model_id = "<model_id>"
#post_url = endpoint + "/formrecognizer/v2.0-preview/custom/models/%s/analyze" % model_id
source = r"C:\Users\Kawar\Documents\CSIRO Plastics Illegal Fishing\Form1.png"
params = {
    "includeTextDetails": True
}

headers = {
    # Request headers
    'Content-Type': 'image/png',
    'Ocp-Apim-Subscription-Key': apim_key,
}
with open(source, "rb") as f:
    data_bytes = f.read()

try:
    resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
    if resp.status_code != 202:
        print("POST analyze failed:\n%s" % json.dumps(resp.json()))
        quit()
    print("POST analyze succeeded:\n%s" % resp.headers)
    get_url = resp.headers["operation-location"]
except Exception as e:
    print("POST analyze failed:\n%s" % str(e))
    quit()

POST analyze failed:
{"error": {"code": "1002", "message": "Train request is either invalid or missing required parameters. Please reference the API reference and retry your request."}}
POST analyze succeeded:
{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'x-envoy-upstream-service-time': '3', 'apim-request-id': 'ac3a9d98-1dc0-4407-b08e-de796983a20c', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'Date': 'Mon, 06 Apr 2020 23:38:16 GMT'}
POST analyze failed:
'operation-location'


In [6]:
n_tries = 15
n_try = 0
wait_sec = 5
max_wait_sec = 60
while n_try < n_tries:
    try:
        resp = get(url = get_url, headers = {"Ocp-Apim-Subscription-Key": apim_key})
        resp_json = resp.json()
        if resp.status_code != 200:
            print("GET analyze results failed:\n%s" % json.dumps(resp_json))
            quit()
        status = resp_json["status"]
        if status == "succeeded":
            print("Analysis succeeded:\n%s" % json.dumps(resp_json,indent=4, sort_keys=True))
            break
        if status == "failed":
            print("Analysis failed:\n%s" % json.dumps(resp_json))
            quit()
        # Analysis still running. Wait and retry.
        time.sleep(wait_sec)
        n_try += 1
        wait_sec = min(2*wait_sec, max_wait_sec)     
    except Exception as e:
        msg = "GET analyze results failed:\n%s" % str(e)
        print(msg)
        quit()
print("Analyze operation did not complete within the allocated time.")

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
GET analyze results failed:
'status'
G

KeyboardInterrupt: 